# LSTM

Implementing LSTM for IMDB data. For data analysis, look at Logistic_Regression.ipynb

In [1]:
import imdb_functions # For easy data load.
import keras
import numpy as np
import pandas as pd
import sklearn as sk

from keras.datasets import imdb # Easier way to implement
from keras.layers import LSTM, Dense, Input, Embedding, Reshape
from keras.models import Model, save_model, load_model
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


In [2]:
(X_train,y_train),(X_test,y_test)=imdb.load_data(num_words=5000)

In [3]:
len(X_train), len(y_train),len(X_test), len(y_test)
# imdb1[0]

(25000, 25000, 25000, 25000)

In [4]:
max_review_length = 500
X_train = pad_sequences(X_train, maxlen=max_review_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_review_length)

In [12]:
max([i.max() for i in X_train])

4999

In [5]:
X_train[0]

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5,  150,
          4,  172,  112,  167,    2,  336,  385,   39,    4,  172, 4536,
       1111,   17,  546,   38,   13,  447,    4,  192,   50,   16,    6,
        147, 2025,   19,   14,   22,    4, 1920, 4613,  469,    4,   22,
         71,   87,   12,   16,   43,  530,   38,   76,   15,   13, 1247,
          4,   22,   17,  515,   17,   12,   16,  626,   18,    2,    5,
         62,  386,   12,    8,  316,    8,  106,    5,    4, 2223,    2,
         16,  480,   66, 3785,   33,    4,  130,   12,   16,   38,  619,
          5,   25,  124,   51,   36,  135,   48,   25, 1415,   33,    6,
         22,   12,  215,   28,   77,   52,    5,   14,  407,   16,   82,
          2,    8,    4,  107,  117,    2,   15,  256,    4,    2,    7,
       3766,    5,  723,   36,   71,   43,  530,  4

In [19]:
# Model setup
input_nodes= Input(shape=(X_train.shape[1],))
# x = Dense(64, activation='relu', kernel_initializer='random_uniform')(input_nodes)
# e = Embedding(output_dim=64, input_dim=X_train.shape[1],embeddings_initializer='glorot_uniform', input_length=10)(input_nodes)
# e = Embedding(5000,
#               16,
# #               weights=[embedding_matrix],
#               input_length=X_train.shape[1],
#               trainable=False)(input_nodes)
# x = Reshape(target_shape=())(input_nodes)
lstm=LSTM(32)(input_nodes)
output_nodes=Dense(1, activation='sigmoid')(lstm)

#Build model
model = Model(inputs=input_nodes, outputs=output_nodes)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

ValueError: Input 0 is incompatible with layer lstm_5: expected ndim=3, found ndim=2

In [20]:
model.fit(X_train, y_train, batch_size=128, epochs=4)

Epoch 1/4
25000/25000 [==============================] - 83s 3ms/step - loss: 0.6928 - acc: 0.5050
Epoch 2/4
25000/25000 [==============================] - 91s 4ms/step - loss: 0.7007 - acc: 0.5034
Epoch 3/4
25000/25000 [==============================] - 81s 3ms/step - loss: 0.6930 - acc: 0.5060
Epoch 4/4
25000/25000 [==============================] - 92s 4ms/step - loss: 0.6928 - acc: 0.5103


In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)


In [ ]:
# Create tensor/model background.
input_nodes = Input(shape=(X_train_vector.shape[1],))
x = Dense(64, activation='relu', kernel_initializer='random_uniform')(input_nodes)
# x = Dense(64, activation='relu')(x)
output_nodes = Dense(1, activation='sigmoid')(x) ##  (input_nodes) if there's no hidden layer

# Create the model with layers specified above.
model = Model(inputs=input_nodes, outputs=output_nodes)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train_vector, y_train, epochs=2)  # starts training